In [1]:
!pip install torch
!pip install accelerate datasets transformers evaluate
import torch, transformers, datasets, accelerate, evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from torch.utils.data import DataLoader
from accelerate.utils.memory import clear_device_cache
from datasets import load_dataset, get_dataset_split_names, get_dataset_config_names

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [2]:
dataset = load_dataset("yelp_review_full")

README.md: 0.00B [00:00, ?B/s]

yelp_review_full/train-00000-of-00001.pa(…):   0%|          | 0.00/299M [00:00<?, ?B/s]

yelp_review_full/test-00000-of-00001.par(…):   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
get_dataset_split_names("yelp_review_full") #task model is going to predict the review quality

['train', 'test']

In [4]:
dataset["train"].features #features of the 'train' dataset

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars']),
 'text': Value('string')}

In [5]:
dataset["train"][17] #a random example from the train dataset

{'label': 4,
 'text': 'Old school.....traditional \\"mom \'n pop\\" quality and perfection. The best fish and chips you\'ll ever enjoy and equally superb fried shrimp. A great out of the way, non-corporate, vestige of Americana. You will love it.'}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
###Create small subsets of data using only 1000 examples for train and evaluation
small_train_dataset = dataset["train"].shuffle(seed = 42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed = 42).select(range(1000))

In [8]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
small_train_dataset = small_train_dataset.map(tokenize_function, batched = True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
metric = evaluate.load("accuracy")


In [12]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predicitions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predicitions, references=labels)


In [13]:
training_args = TrainingArguments(
    report_to="none",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    warmup_ratio=0.1,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    logging_dir='./logs',
    logging_steps=10,
    output_dir="test_trainer",
    eval_strategy="epoch"
)


In [14]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.584800,1.519277,0.359000
2,1.323100,1.221848,0.516000
3,1.024000,1.037666,0.576000
4,0.890200,0.980665,0.608000
5,0.728300,0.969868,0.601000


TrainOutput(global_step=160, training_loss=1.1420563280582428, metrics={'train_runtime': 648.0195, 'train_samples_per_second': 7.716, 'train_steps_per_second': 0.247, 'total_flos': 1315590712320000.0, 'train_loss': 1.1420563280582428, 'epoch': 5.0})

In [15]:
#the loss should be decreasing
#validation set hasnt been seen by the model, we see the loss, validation loss hould increase
#can experiement the hyperamets such as increasing the number of empoch or learning rate and see the results